In [1]:
import pandas as pd
import requests
import os
import json

## Exploring EV data from evdatahub

In [2]:

CURRENT_PATH = os.getcwd()

In [3]:
DATA_PATH = os.path.join(CURRENT_PATH,"data")

In [4]:
reg_df = pd.DataFrame()

In [8]:
ev_reg_files = ['co_ev_registrations_public.xlsx',
 'ct_ev_registrations.xlsx',
 'fl_ev_registrations.xlsx',
 'mi_ev_registrations_public.xlsx',
 'mn_ev_registrations_public.xlsx',
 'nj_ev_registrations_public.xlsx',
 'ny_ev_registrations_public.xlsx',
 'or_ev_registrations_public.xlsx',
 'tx_ev_registrations_public.xlsx',
 'va_ev_registrations_public.xlsx',
 'vt_ev_registrations_public.xlsx',
 'wa_ev_registrations_public.xlsx',
 'wi_ev_registrations_public.xlsx']


In [9]:
for f in ev_reg_files:
    file_path = os.path.join(DATA_PATH,f)
    temp_df = pd.read_excel(file_path)
    temp_df['file_name'] = f
#     temp_df['state'] = file_path.split("_")[-4].split("\\")[-1] ## getting the name of the state from the file name
    reg_df = pd.concat([reg_df,temp_df])

C:\Users\SMaharana\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [10]:
reg_df['file_name'].value_counts()

ny_ev_registrations_public.xlsx    956830
wa_ev_registrations_public.xlsx    429192
nj_ev_registrations_public.xlsx     91143
or_ev_registrations_public.xlsx     76342
tx_ev_registrations_public.xlsx     74143
fl_ev_registrations.xlsx            67209
va_ev_registrations_public.xlsx     28039
mi_ev_registrations_public.xlsx     27518
co_ev_registrations_public.xlsx     22751
mn_ev_registrations_public.xlsx     16177
ct_ev_registrations.xlsx            10569
wi_ev_registrations_public.xlsx      6380
vt_ev_registrations_public.xlsx      6196
Name: file_name, dtype: int64

In [11]:
reg_df['state'] = reg_df['file_name'].apply(lambda x: x.split("_")[0])

In [12]:
reg_df.head()

,County,County GEOID,DMV ID,Registration Expiration Date,Registration Valid Date,VIN Model Year,VIN Prefix,Vehicle Name,ZIP Code,file_name,state
0,NaN,NaN,1,2020-08-31 00:00:00,2019-08-07,K,5YJ3E1EA,NaN,80003,co_ev_registrations_public.xlsx,co
1,NaN,NaN,1,2020-08-31 00:00:00,2019-08-07,K,5YJ3E1EA,NaN,80241,co_ev_registrations_public.xlsx,co
2,NaN,NaN,1,2020-08-31 00:00:00,2019-08-07,G,1N4BZ0CP,NaN,80212,co_ev_registrations_public.xlsx,co
3,NaN,NaN,1,2020-08-31 00:00:00,2019-08-07,E,5YJSA1H1,NaN,80210,co_ev_registrations_public.xlsx,co
4,NaN,NaN,1,2020-07-31 00:00:00,2019-08-07,C,5YJSA1CN,NaN,80304,co_ev_registrations_public.xlsx,co


### Combining all imported data

In [42]:
reg_df.to_csv('data/registration_data_all_states.csv')

In [66]:
import datetime as dt
reg_df['Registration Expiration Date'] = pd.to_datetime(reg_df['Registration Expiration Date'],errors = 'coerce')
reg_df['Registration Valid Date'] = pd.to_datetime(reg_df['Registration Valid Date'],errors = 'coerce')

In [88]:
reg_df['Exp_year'] = reg_df['Registration Expiration Date'].dt.year
reg_df['Valid_year'] = reg_df['Registration Valid Date'].dt.year

In [89]:
reg_df.head()

,County,County GEOID,DMV ID,Registration Expiration Date,Registration Valid Date,VIN Model Year,VIN Prefix,Vehicle Name,ZIP Code,file_name,state,Exp_year,Valid_year
0,NaN,NaN,1,2020-08-31,2019-08-07,K,5YJ3E1EA,NaN,80003,co_ev_registrations_public.xlsx,co,2020.0,2019.0
1,NaN,NaN,1,2020-08-31,2019-08-07,K,5YJ3E1EA,NaN,80241,co_ev_registrations_public.xlsx,co,2020.0,2019.0
2,NaN,NaN,1,2020-08-31,2019-08-07,G,1N4BZ0CP,NaN,80212,co_ev_registrations_public.xlsx,co,2020.0,2019.0
3,NaN,NaN,1,2020-08-31,2019-08-07,E,5YJSA1H1,NaN,80210,co_ev_registrations_public.xlsx,co,2020.0,2019.0
4,NaN,NaN,1,2020-07-31,2019-08-07,C,5YJSA1CN,NaN,80304,co_ev_registrations_public.xlsx,co,2020.0,2019.0


In [43]:
## get all states
unique_state_list = reg_df['state'].unique().tolist()

### Can we group by Exp date?

In [125]:
grouped_by_exp_year = pd.DataFrame({'count' : reg_df.groupby(['state','Exp_year']).size()}).reset_index()

In [128]:
grouped_by_exp_year[grouped_by_exp_year['Exp_year']>=2018].groupby(['state'])['count'].sum()

state
co     22746
ct      7951
mn     15996
ny    794311
va     27489
wa     60317
Name: count, dtype: int64

In [129]:
grouped_by_val_year = pd.DataFrame({'count' : reg_df.groupby(['state','Valid_year']).size()}).reset_index()

In [131]:
grouped_by_val_year[grouped_by_val_year['Valid_year']>=2018].groupby(['state'])['count'].sum()

state
co     22749
ct      6210
fl     67209
mi     17827
mn     16087
nj     67140
ny    507246
or     23312
tx     74143
va     13613
vt      2090
wa    340255
wi      6380
Name: count, dtype: int64

### Can we pick the max reg date

In [133]:
for state in unique_state_list:
    try:
        ## filter dataframe by state
        df = reg_df[reg_df['state'] ==state]
        ## get max reg date for each state
        max_reg_date = df['Registration Valid Date'].max()
        ## filter df to remove duplicates
        temp_df = df[df['Registration Valid Date']>=max_reg_date]
        print (state, "-->", temp_df.shape[0])
    except:
        print ("Skipping..", state)

co --> 76
ct --> 6210
fl --> 37776
mi --> 72
mn --> 1
nj --> 26642
ny --> 2
or --> 22107
tx --> 26321
va --> 2
vt --> 1
wa --> 53431
wi --> 3585


## Cleaning by States

In [140]:
unique_state_list

['co', 'ct', 'fl', 'mi', 'mn', 'nj', 'ny', 'or', 'tx', 'va', 'vt', 'wa', 'wi']

Several states dont have data for 2019 and also beyond 2016. To maintain consistency, we will just take the data for 2017-18.

### Colorado

In [224]:
reg_df[reg_df['state']=='co'].shape

(22751, 13)

In [225]:
reg_df[reg_df['state']=='co']['Valid_year'].value_counts()

2019.0    14763
2018.0     7986
2016.0        2
Name: Valid_year, dtype: int64

In [249]:
col_df = reg_df[(reg_df['state']=='co') & (reg_df['Valid_year'].isin([2017,2018]))]

In [250]:
col_df.head()


,County,County GEOID,DMV ID,Registration Expiration Date,Registration Valid Date,VIN Model Year,VIN Prefix,Vehicle Name,ZIP Code,file_name,state,Exp_year,Valid_year
14763,NaN,NaN,1,2019-12-31,2018-12-31,J,5YJSA1E2,NaN,80138,co_ev_registrations_public.xlsx,co,2019.0,2018.0
14764,NaN,NaN,1,2019-12-31,2018-12-31,J,5YJXCAE2,NaN,81620,co_ev_registrations_public.xlsx,co,2019.0,2018.0
14765,NaN,NaN,1,2019-12-31,2018-12-31,C,1G1RD6E4,NaN,80538,co_ev_registrations_public.xlsx,co,2019.0,2018.0
14766,NaN,NaN,1,2019-11-30,2018-12-31,G,WBY2Z2C5,NaN,80915,co_ev_registrations_public.xlsx,co,2019.0,2018.0
14767,NaN,NaN,1,2019-12-31,2018-12-31,F,5YJSA1S2,NaN,80228,co_ev_registrations_public.xlsx,co,2019.0,2018.0


In [236]:
col_df['ZIP Code'].nunique()

308

THis seems to match with the number in https://www.atlasevhub.com/materials/state-ev-registration-data/

In [148]:
col_df = df

### Connecticut

In [237]:
reg_df[reg_df['state']=='ct'].shape

(10569, 13)

In [238]:
reg_df[reg_df['state']=='ct']['Valid_year'].value_counts()

2018.0    6210
2015.0    1970
2016.0    1765
2014.0     511
2013.0      89
2012.0      23
2011.0       1
Name: Valid_year, dtype: int64

In [251]:
con_df = reg_df[(reg_df['state']=='ct') & (reg_df['Valid_year'].isin([2017,2018]))]

In [252]:
con_df.head()

,County,County GEOID,DMV ID,Registration Expiration Date,Registration Valid Date,VIN Model Year,VIN Prefix,Vehicle Name,ZIP Code,file_name,state,Exp_year,Valid_year
0,NaN,NaN,2,2018-12-31,2018-01-04,F,5YJSA1E2,NaN,6878,ct_ev_registrations.xlsx,ct,2018.0,2018.0
1,NaN,NaN,2,2018-12-31,2018-01-04,C,1G1RB6E4,NaN,6111,ct_ev_registrations.xlsx,ct,2018.0,2018.0
2,NaN,NaN,2,2018-12-31,2018-01-04,G,YV4BC0PL,NaN,6840,ct_ev_registrations.xlsx,ct,2018.0,2018.0
3,NaN,NaN,2,2018-12-31,2018-01-04,G,1G1RD6S5,NaN,6088,ct_ev_registrations.xlsx,ct,2018.0,2018.0
4,NaN,NaN,2,2018-12-31,2018-01-04,F,1FADP5CU,NaN,60422,ct_ev_registrations.xlsx,ct,2018.0,2018.0


In [253]:
con_df['ZIP Code'].nunique()

842

### Michigan

In [244]:
reg_df[reg_df['state']=='mi'].shape

(27518, 13)

In [245]:
reg_df[reg_df['state']=='mi']['Valid_year'].value_counts()

2019.0    10779
2017.0     7992
2018.0     7048
2016.0      751
2015.0      554
2014.0      391
2013.0        3
Name: Valid_year, dtype: int64

In [254]:
mi_df = reg_df[(reg_df['state']=='mi') & (reg_df['Valid_year'].isin([2017,2018]))]

In [255]:
mi_df.shape

(15040, 13)

In [256]:
mi_df['ZIP Code'].nunique()

821

### Minnesota

In [257]:
reg_df[reg_df['state']=='mn'].shape

(16177, 13)

In [258]:
reg_df[reg_df['state']=='mn']['Valid_year'].value_counts()

2018.0    12946
2019.0     3141
2017.0       38
2016.0       12
2008.0       10
2012.0        8
2013.0        7
2015.0        4
2011.0        4
2009.0        3
2014.0        1
Name: Valid_year, dtype: int64

In [259]:
mn_df = reg_df[(reg_df['state']=='mn') & (reg_df['Valid_year'].isin([2017,2018]))]

In [260]:
mn_df.shape

(12984, 13)

In [261]:
mn_df['ZIP Code'].nunique()

507

### New Jersey

In [262]:
reg_df[reg_df['state']=='nj'].shape

(91143, 13)

In [263]:
reg_df[reg_df['state']=='nj']['Valid_year'].value_counts()

2018.0    40498
2019.0    26642
2017.0    24003
Name: Valid_year, dtype: int64

Seems to be duplicates.. ignoring NJ

### New York

In [264]:
reg_df[reg_df['state']=='ny'].shape

(956830, 13)

In [265]:
reg_df[reg_df['state']=='ny']['Valid_year'].value_counts()

2018.0    366119
2017.0    257046
2019.0    141127
2016.0     93205
2015.0     40830
2014.0     36996
2013.0     10798
2012.0      8357
2011.0      2352
Name: Valid_year, dtype: int64

Duplicates.. excluding this as well

### Washington

In [266]:
reg_df[reg_df['state']=='wa'].shape

(429192, 13)

In [267]:
reg_df[reg_df['state']=='wa']['Valid_year'].value_counts()

2019.0    299839
2018.0     40416
2017.0     28300
2016.0     20970
2015.0     15891
2014.0     11934
2013.0      7619
2012.0      2948
2011.0      1225
2010.0        50
Name: Valid_year, dtype: int64

Duplicates..

 Got data from https://catalog.data.gov/dataset/electric-vehicle-population-data

In [284]:
wa_df = pd.read_csv('data/WA_Electric_Vehicle_Population_Data.csv',usecols =['VIN (1-10)','ZIP Code','State'])

In [285]:
wa_df.head()

,VIN (1-10),State,ZIP Code
0,1G1FW6S03H,WA,98107
1,3FA6P0SU8F,WA,98370
2,1N4AZ1CP1J,WA,98360
3,1N4AZ0CP5D,WA,98006
4,JTDKN3DP0D,WA,98034


In [286]:
wa_df.columns = ['VIN Prefix','state','ZIP Code']

### Texas

In [289]:
reg_df[reg_df['state']=='tx'].shape

(74143, 13)

In [290]:
reg_df[reg_df['state']=='tx']['Valid_year'].value_counts()

2019.0    74143
Name: Valid_year, dtype: int64

In [291]:
tx_df = reg_df[(reg_df['state']=='mn') & (reg_df['Valid_year'].isin([2019]))]

Since Texas doesnt have 2017-18 data.. taking 2019(Should we?)

### Oregon

In [292]:
reg_df[reg_df['state']=='or'].shape

(76342, 13)

In [293]:
reg_df[reg_df['state']=='or']['Valid_year'].value_counts()

2018.0    23312
2017.0    18444
2016.0    13445
2015.0     8929
2014.0     6128
2013.0     3847
2012.0     1665
2011.0      572
Name: Valid_year, dtype: int64

In [294]:
or_df = reg_df[(reg_df['state']=='or') & (reg_df['Valid_year'].isin([2017,2018]))]

In [295]:
or_df.shape

(41756, 13)

### Virginia

In [296]:
reg_df[reg_df['state']=='va'].shape

(28039, 13)

In [297]:
reg_df[reg_df['state']=='va']['Valid_year'].value_counts()

2018.0    13611
2017.0    11458
2016.0     2548
2015.0      283
2014.0       25
2013.0       23
2011.0       21
2012.0       16
2006.0       14
2009.0       14
2007.0        8
2010.0        6
2002.0        4
2005.0        2
2004.0        2
2003.0        2
2019.0        2
Name: Valid_year, dtype: int64

In [298]:
va_df = reg_df[(reg_df['state']=='va') & (reg_df['Valid_year'].isin([2017,2018]))]

### Vermont

In [299]:
reg_df[reg_df['state']=='vt'].shape

(6196, 13)

In [300]:
reg_df[reg_df['state']=='vt']['Valid_year'].value_counts()

2017.0    2060
2018.0    1974
2016.0    1104
2015.0     401
2013.0     187
2014.0     160
2019.0     116
2012.0     104
2003.0      44
2011.0      20
2008.0      10
2007.0       6
2009.0       4
2010.0       2
2006.0       2
2005.0       2
Name: Valid_year, dtype: int64

In [301]:
vt_df = reg_df[(reg_df['state']=='vt') & (reg_df['Valid_year'].isin([2017,2018]))]

## Finalizing..

### Combining all cleaned data

In [302]:
df = pd.concat([col_df,con_df,mi_df,mn_df,wa_df,tx_df,or_df,va_df,vt_df])

C:\Users\SMaharana\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [303]:
df.shape

(170198, 13)

### Checking consistencies

In [307]:
df = df[df['ZIP Code'].isnull()!=True] ## Removed rows with NULL ZIP Codes

In [308]:
df.shape

(145108, 13)

In [309]:
df.to_csv('data/ev_data.csv')